In [17]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget
if 'NOTEBOOK_INITIATED_FLAG' not in globals():
    NOTEBOOK_INITIATED_FLAG = True
    %cd ..
import os 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image

import examplesetup
import lib.deconvolution as deconvolution
import lib.utils as utils
import lib.xtrace as xtrace
import lib.datagen as datagen

from numpy.random import default_rng
rng = default_rng()

import tensorflow as tf
from tensorflow import keras
import numpy as np
import itertools

model_paths = {
    #"Noise": 'deconvolve_smallnoise_randgrid_oneloc.h5',
    "Hits - UNIF grid": 'deconvolve_hits_grid_oneloc.h5',
    "Hits - RAND grid": 'deconvolve_hits_randomgrid_oneloc.h5',
    "Hits - RAND grid 2": 'deconvolve_hits_randomgrid_oneloc_2.h5',
}
test_image_paths = {
    "Image 1": 'test_data_9.npy',
    "Image 2": 'test_data_0.npy',
}


models = {k:keras.models.load_model(f'trained_networks/{v}') for k,v in model_paths.items()}
test_images = {k:np.load(f'example_files/{v}')/1e6 for k,v in test_image_paths.items()}

#distorted_img_syn, img_syn = datagen.get_synthetic_data_pair()

percent = 99.5
perc = np.percentile(np.maximum(*test_images.values()), percent)

n, m = len(models) + 1, len(test_images)
fig, axs = plt.subplots(m, n, figsize=(2.2*n, 2.2*m), sharex=True, sharey=True)
for i, (m_name, model) in enumerate(models.items()):
    for j, (i_name, image) in enumerate(test_images.items()):
        recovered_img = model(image[np.newaxis,:,:], training=False).numpy()[0,:,:,0]
        axs[j, i + 1].imshow(recovered_img, vmax=perc, vmin=0)
        axs[j, i + 1].set_title(f'{m_name}')
for j, (i_name, image) in enumerate(test_images.items()):
    axs[j, 0].imshow(image, vmax=perc, vmin=0)
    axs[j, 0].set_title(f'{i_name}')

plt.tight_layout()
plt.show()
#Add local energy loss constraint? (ex: 10x10 area should contain about the same ammount of energy)
#Add positivity loss constraint?
#Try the Pecent contraint with lower randomness value -> trains on more realistic schenarios

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …